In [2]:
!pip install NLTK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
# Import necessary libraries
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
# Load the dataset
df = pd.read_csv('data_bersih.csv')
df

,title,label,label_score
0,masuk radar pilwalkot medan menantu jokowi tem...,non-clickbait,0
1,malaysia sudut ri isu kabut asap invasi babi,non-clickbait,0
2,viral driver ojol bekas pesan makan pakai sepeda,clickbait,1
3,kemensos salur rp m korban rusuh sosial papua,non-clickbait,0
4,kait mayat bayi enas tangerang pria tangkap po...,non-clickbait,0
...,...,...,...
14995,tolak ruu tanah ribu tani gelar aksi istanadpr...,non-clickbait,0
14996,niat momong program hamil fedi nuril pengin ya...,clickbait,1
14997,edar isu internet papua blokir telkom warga pe...,non-clickbait,0
14998,txt comeback soobin aku gatal pamer spoiler,clickbait,1


In [10]:
print(df.isnull().sum())

title          0
label          0
label_score    0
dtype: int64


In [11]:
# Pisahkan atribut dan label
X = df['title']
y = df['label_score']

In [12]:
X

0        masuk radar pilwalkot medan menantu jokowi tem...
1             malaysia sudut ri isu kabut asap invasi babi
2         viral driver ojol bekas pesan makan pakai sepeda
3            kemensos salur rp m korban rusuh sosial papua
4        kait mayat bayi enas tangerang pria tangkap po...
                               ...                        
14995    tolak ruu tanah ribu tani gelar aksi istanadpr...
14996    niat momong program hamil fedi nuril pengin ya...
14997    edar isu internet papua blokir telkom warga pe...
14998          txt comeback soobin aku gatal pamer spoiler
14999    emmy awards peter dinklage cetak rekor berkat ...
Name: title, Length: 15000, dtype: object

In [13]:
y

0        0
1        0
2        1
3        0
4        0
        ..
14995    0
14996    1
14997    0
14998    1
14999    0
Name: label_score, Length: 15000, dtype: int64

In [15]:
# membagi dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# mengubah fitur menjadi vektor tf-idf
tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

# Train model
best_score = 0
best_k = 0
for k in range(1, 12, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5)
    score = scores.mean()
    if score > best_score:
        best_score = score
        best_k = k
    print(f"k = {k}, Accuracy = {score}")

print(f"Best k: {best_k}")

knn = KNeighborsClassifier(n_neighbors=best_k)
knn.fit(X_train, y_train)

# Evaluate model
y_pred = knn.predict(X_test)

k = 1, Accuracy = 0.6448333333333334
k = 3, Accuracy = 0.6640833333333334
k = 5, Accuracy = 0.6812500000000001
k = 7, Accuracy = 0.6879166666666666
k = 9, Accuracy = 0.69325
k = 11, Accuracy = 0.6965
Best k: 11


In [16]:
print(confusion_matrix(y_test, y_pred))

[[1463  335]
 [ 519  683]]


In [17]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.81      0.77      1798
           1       0.67      0.57      0.62      1202

    accuracy                           0.72      3000
   macro avg       0.70      0.69      0.69      3000
weighted avg       0.71      0.72      0.71      3000



In [20]:
import joblib
# save model
filename = 'model.joblib'
pickle.dump(knn, open(filename, 'wb'))

In [21]:
# save vectorizer
filename = 'vectorizer.joblib'
pickle.dump(tfidf, open(filename, 'wb'))

In [ ]:
# load model
loaded_model = pickle.load(open(filename, 'rb'))